In [ ]:
pip install split-folders

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import splitfolders
basic_data = '/content/drive/MyDrive/project'
splitfolders.ratio(basic_data , output = "output_dataset" , seed=1337 , ratio=(0.8,0.1,0.1) , group_prefix=None)

Copying files: 15630 files [14:06, 18.46 files/s] 


In [ ]:
train_path = '/content/output_dataset/train'
validation_path ='/content/output_dataset/val'
test_path = '/content/output_dataset/test'

In [ ]:

base_model = Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Change here
model = Model(inputs=base_model.input, outputs=predictions)

# Compiling the CNN
opt = Adam(lr =0.01)
model.compile(optimizer = opt , loss = 'binary_crossentropy', metrics = ['accuracy'])

83683744/83683744 [==============================] - 0s 0us/step


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory('/content/output_dataset/train',
                                                    target_size=(256,256),
                                                    batch_size=64,
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory('/content/output_dataset/val',
                                                              target_size=(256 , 256),
                                                              batch_size=64,
                                                              class_mode='binary')

Found 12503 images belonging to 2 classes.
Found 1562 images belonging to 2 classes.


In [ ]:
history = model.fit(train_generator,
          steps_per_epoch = len(train_generator),
          epochs = 15,
          validation_data=validation_generator,
          validation_steps = len(validation_generator))

Epoch 1/15
196/196 [==============================] - 399s 2s/step - loss: 0.3623 - accuracy: 0.8411 - val_loss: 0.3189 - val_accuracy: 0.8956
Epoch 2/15
196/196 [==============================] - 353s 2s/step - loss: 0.2492 - accuracy: 0.9008 - val_loss: 0.2888 - val_accuracy: 0.8988
Epoch 3/15
196/196 [==============================] - 352s 2s/step - loss: 0.2095 - accuracy: 0.9175 - val_loss: 0.2401 - val_accuracy: 0.9033
Epoch 4/15
196/196 [==============================] - 357s 2s/step - loss: 0.1624 - accuracy: 0.9391 - val_loss: 0.9118 - val_accuracy: 0.6671
Epoch 5/15
196/196 [==============================] - 357s 2s/step - loss: 0.1482 - accuracy: 0.9447 - val_loss: 0.2970 - val_accuracy: 0.8969
Epoch 6/15
196/196 [==============================] - 364s 2s/step - loss: 0.1202 - accuracy: 0.9559 - val_loss: 0.3309 - val_accuracy: 0.8988
Epoch 7/15
196/196 [==============================] - 361s 2s/step - loss: 0.0979 - accuracy: 0.9642 - val_loss: 0.1142 - val_accuracy: 0.9577

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('/content/output_dataset/test',
                                            target_size = (256,256),
                                            batch_size = 64,
                                            class_mode = 'binary',
                                            shuffle = False)

Found 1565 images belonging to 2 classes.


In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print('test accuracy:',test_acc )
print('test loss:',test_loss)

25/25 [==============================] - 17s 664ms/step - loss: 0.1356 - accuracy: 0.9629
test accuracy: 0.9629393219947815
test loss: 0.13557998836040497


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
y_pred = model.predict(test_generator, workers = 0)
# For binary classification, we round the predictions to get the class
y_pred_classes = y_pred.round().astype(int)

y_true = test_generator.classes[test_generator.index_array]
class_names = list(test_generator.class_indices.keys())


25/25 [==============================] - 14s 552ms/step


In [ ]:
# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

Confusion Matrix:
[[732  32]
 [ 26 775]]


In [ ]:
# Print the classification report
print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))

Classification Report:
              precision    recall  f1-score   support

    glaucoma       0.97      0.96      0.96       764
      normal       0.96      0.97      0.96       801

    accuracy                           0.96      1565
   macro avg       0.96      0.96      0.96      1565
weighted avg       0.96      0.96      0.96      1565



In [ ]:
model.save('XCEPTION_Classifier.h5')

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# Load your trained model
model = load_model('/content/XCEPTION_Classifier.h5')

# Load the image you want to test
img_path = '/content/drive/MyDrive/gloucoma.jpg'
img = image.load_img(img_path, target_size=(256, 256))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Expand dimensions so the model can accept the image
img_array = np.expand_dims(img_array, axis=0)

# Preprocess the image according to your model's requirements
img_array /= 255.  # if your model expects images scaled between 0 and 1

# Use the model to predict the image's class
preds = model.predict(img_array)

# For binary classification, we round the predictions to get the class
pred_class = preds.round().astype(int)

# Print the predicted class
print('Predicted class:', pred_class[0])

1/1 [==============================] - 1s 733ms/step
Predicted class: [0]


In [ ]:
# Assuming class_names is your list of class names
class_names = ['glaucoma', 'normal']

# Get the class name
class_name = class_names[pred_class[0][0]]

# Print the class name
print('Predicted class name:', class_name)

Predicted class name: glaucoma


In [ ]:
# Load the image you want to test
img_path = '/content/drive/MyDrive/normal.jpg'
img = image.load_img(img_path, target_size=(256, 256))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Expand dimensions so the model can accept the image
img_array = np.expand_dims(img_array, axis=0)

# Preprocess the image according to your model's requirements
img_array /= 255.  # if your model expects images scaled between 0 and 1

# Use the model to predict the image's class
preds = model.predict(img_array)

# For binary classification, we round the predictions to get the class
pred_class = preds.round().astype(int)

# Print the predicted class
print('Predicted class:', pred_class[0])

1/1 [==============================] - 0s 23ms/step
Predicted class: [1]


In [ ]:
# Assuming class_names is your list of class names
class_names = ['glaucoma', 'normal']

# Get the class name
class_name = class_names[pred_class[0][0]]

# Print the class name
print('Predicted class name:', class_name)

Predicted class name: normal
